In [1]:
import pandas as pd
import re

In [2]:

bank_statement_df = pd.read_excel(r"D:\NPS_phase1\BANK STMT PHASE 4\Bank_muktinath.xlsx", skiprows=1)


In [3]:
bank_statement_df.head()

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number
0,1,91520201002,-22771.0,D,GMEREMIT#GME-211434,48903,NPS-IF-3130622,2023-08-28T12:55:00,NaN
1,2,91520201002,-60000.0,D,CITYREMIT#CEMTAFT 1,48902,NPS-IF-3130606,2023-08-28T12:52:00,NaN
2,3,91520201002,-12000.0,D,IPAY211698787,48901,FTMS-181399,2023-08-28T12:44:00,NaN
3,4,91520201002,-27371.1,D,PRABHUMONEY#PMT9175,48837,NPS-IF-3127669,2023-08-28T00:41:00,NaN
4,5,91520201002,-105000.0,D,GMEREMIT#GME-211201,48900,NPS-IF-3130369,2023-08-28T12:35:00,NaN


In [19]:
for index, row in bank_statement_df.iterrows():
    # print(bank_statement_df.iloc[index]['Transaction ID'])
    for roww in (str(row['Desc2']) ,str( row['Desc3'])):
        id = ''
        if "NPS-IF-" in roww:
            id = re.findall(r'[0-9]{7}',roww)
            bank_statement_df.loc[index, 'Transaction Id'] = id[0]
            break
        
        
        elif "10000" in roww:
            id = re.findall(r'10*[0-9]{7}',roww)
            bank_statement_df.loc[index, 'Transaction Id'] = id[0].replace("10000","")
            break
        
        elif "FTMS-" in roww:
            id = re.findall(r'[0-9]{6}',roww)
            bank_statement_df.loc[index, 'Transaction Id'] = id[0]

In [20]:
bank_statement_df


,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction Id,Transaction ID
0,1,91520201002,-22771.0,D,GMEREMIT#GME-211434,48903,NPS-IF-3130622,2023-08-28T12:55:00,NaN,3130622,3130622
1,2,91520201002,-60000.0,D,CITYREMIT#CEMTAFT 1,48902,NPS-IF-3130606,2023-08-28T12:52:00,NaN,3130606,3130606
2,3,91520201002,-12000.0,D,IPAY211698787,48901,FTMS-181399,2023-08-28T12:44:00,NaN,181399,181399
3,4,91520201002,-27371.1,D,PRABHUMONEY#PMT9175,48837,NPS-IF-3127669,2023-08-28T00:41:00,NaN,3127669,3127669
4,5,91520201002,-105000.0,D,GMEREMIT#GME-211201,48900,NPS-IF-3130369,2023-08-28T12:35:00,NaN,3130369,3130369
...,...,...,...,...,...,...,...,...,...,...,...
142,143,91520201002,-14125.0,D,IPAY11264937035,48906,FTMS-181428,2023-08-28T13:30:00,NaN,181428,181428
143,144,91520201002,-26790.0,D,GMEREMIT#GME-211995,48905,NPS-IF-3130828,2023-08-28T13:25:00,NaN,3130828,3130828
144,145,91520201002,-35000.0,D,GMEREMIT#GME-R21788,48838,NPS-IF-3127707,2023-08-28T01:21:00,NaN,3127707,3127707
145,146,91520201002,-3500.0,D,NICASIA#100,48904,NPS-IF-3130761,2023-08-28T13:15:00,NaN,3130761,3130761


In [21]:
bank_statement_df['Transaction Id'].count()

147

In [22]:
def standard_format_phase_4(row):
        """
        Determine transaction mode and amount from bank statement data.

        Parameters:
            row (Series): A row of bank statement data.

        Returns:
            Series: Mode (CR/DR) and amount.
        """
        if row['Txn Type'] == 'D' :
            mode = 'DR'
            amount = abs(row['Amount'])
        elif row['Txn Type'] == 'C':
            mode = 'CR'
            amount = row['Amount']
        else:
            mode = None
            amount = None
        return pd.Series([mode, amount], index=['Mode', 'Amount'])

In [23]:
bank_statement_df[['Mode', 'Amount']] = bank_statement_df.apply(standard_format_phase_4, axis=1)


In [25]:
bank_statement_df

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction Id,Transaction ID,Mode
0,1,91520201002,22771.0,D,GMEREMIT#GME-211434,48903,NPS-IF-3130622,2023-08-28T12:55:00,NaN,3130622,3130622,DR
1,2,91520201002,60000.0,D,CITYREMIT#CEMTAFT 1,48902,NPS-IF-3130606,2023-08-28T12:52:00,NaN,3130606,3130606,DR
2,3,91520201002,12000.0,D,IPAY211698787,48901,FTMS-181399,2023-08-28T12:44:00,NaN,181399,181399,DR
3,4,91520201002,27371.1,D,PRABHUMONEY#PMT9175,48837,NPS-IF-3127669,2023-08-28T00:41:00,NaN,3127669,3127669,DR
4,5,91520201002,105000.0,D,GMEREMIT#GME-211201,48900,NPS-IF-3130369,2023-08-28T12:35:00,NaN,3130369,3130369,DR
...,...,...,...,...,...,...,...,...,...,...,...,...
142,143,91520201002,14125.0,D,IPAY11264937035,48906,FTMS-181428,2023-08-28T13:30:00,NaN,181428,181428,DR
143,144,91520201002,26790.0,D,GMEREMIT#GME-211995,48905,NPS-IF-3130828,2023-08-28T13:25:00,NaN,3130828,3130828,DR
144,145,91520201002,35000.0,D,GMEREMIT#GME-R21788,48838,NPS-IF-3127707,2023-08-28T01:21:00,NaN,3127707,3127707,DR
145,146,91520201002,3500.0,D,NICASIA#100,48904,NPS-IF-3130761,2023-08-28T13:15:00,NaN,3130761,3130761,DR


In [26]:
bank_statement_df['Date'] = pd.to_datetime(bank_statement_df['Date'], format="%Y-%m-%dT%H:%M:%S").dt.date

In [27]:
bank_statement_df

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction Id,Transaction ID,Mode
0,1,91520201002,22771.0,D,GMEREMIT#GME-211434,48903,NPS-IF-3130622,2023-08-28,NaN,3130622,3130622,DR
1,2,91520201002,60000.0,D,CITYREMIT#CEMTAFT 1,48902,NPS-IF-3130606,2023-08-28,NaN,3130606,3130606,DR
2,3,91520201002,12000.0,D,IPAY211698787,48901,FTMS-181399,2023-08-28,NaN,181399,181399,DR
3,4,91520201002,27371.1,D,PRABHUMONEY#PMT9175,48837,NPS-IF-3127669,2023-08-28,NaN,3127669,3127669,DR
4,5,91520201002,105000.0,D,GMEREMIT#GME-211201,48900,NPS-IF-3130369,2023-08-28,NaN,3130369,3130369,DR
...,...,...,...,...,...,...,...,...,...,...,...,...
142,143,91520201002,14125.0,D,IPAY11264937035,48906,FTMS-181428,2023-08-28,NaN,181428,181428,DR
143,144,91520201002,26790.0,D,GMEREMIT#GME-211995,48905,NPS-IF-3130828,2023-08-28,NaN,3130828,3130828,DR
144,145,91520201002,35000.0,D,GMEREMIT#GME-R21788,48838,NPS-IF-3127707,2023-08-28,NaN,3127707,3127707,DR
145,146,91520201002,3500.0,D,NICASIA#100,48904,NPS-IF-3130761,2023-08-28,NaN,3130761,3130761,DR
